# Hands-on: Prompting using LangChain

## Overview

In this hands-on, you will learn about prompting using LangChain, a framework for building LLM applications.
You will learn about:
1. Simple Prompt to LLM using LangChain
2. Zero-Shot Prompt and Few-Shot Prompt using Prompt Template
3. Sequential Prompts using Simple Sequential Chain

## 1. Simple Prompt to LLM (Manually created Prompt Template)
- Basic use case of sending prompts to LLM in watsonx (without using Langchain). 
- In this example, we are sending a simple prompt directly to the LLM model (Google flan-ul2).

In [ ]:
# Install libraries
!pip install chromadb==0.4.2
!pip install langchain==0.0.312
!pip install langchain --upgrade
!pip install flask-sqlalchemy --user
!pip install pypdf 
!pip install sentence-transformers
!pip install langchain_openai
!pip install -U langchain-community
!pip install -U langchain-ibm

In [ ]:
# Import libraries
import os
import warnings

#from dotenv import load_dotenv
from time import sleep
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from langchain import PromptTemplate # Langchain Prompt Template
from langchain.chains import LLMChain, SimpleSequentialChain # Langchain Chains
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator # Vectorize db index with chromadb
from langchain.embeddings import HuggingFaceEmbeddings # For using HuggingFace embedding models
from langchain.text_splitter import CharacterTextSplitter # Text splitter

warnings.filterwarnings("ignore")

In [ ]:
# Set IBM Cloud API key and Project ID
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com"
api_key = "<YOUR IBM CLOUD API KEY HERE>"
project_id = "<YOUR PROJECT ID HERE>"

if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("One or more environment variables are missing!")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

In [ ]:
# Initialize the watsonx model
params = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.TEMPERATURE: 0.2,
    GenParams.TOP_P: 1,
    GenParams.TOP_K: 25,
    GenParams.REPETITION_PENALTY: 1.0,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 20
}

llm_model = Model(
    model_id="google/flan-ul2",
    params=params,
    credentials=creds,
    project_id=project_id
)

print("Done initializing LLM.")

In [ ]:
# Send a simple prompt to model
countries = ["France", "Japan", "Australia"]

try:
  for country in countries:
    question = f"What is the capital of {country}"
    res = llm_model.generate_text(question)
    print(f"The capital of {country} is {res.capitalize()}")
except Exception as e:
  print(e)

## 2. Zero-Shot Prompt and Few-Shot Prompt using LangChain's Prompt Template
- Real use case can be more complex. Instead of sending plain prompts to LLM, we are using Langchain Prompt Template. 
- In this example, we are using Langchain Prompt Template to send prompt to the LLM model (Google flan-ul2).
- Advantags of using Prompt Template:
    1. **Modularity**: With a prompt template, you can define a structured template once and reuse it with different input variables. This makes your code more modular and easier to maintain.
    2. **Dynamic Input**: Prompt templates allow for dynamic input variables, such as "country" in this example. This means you can easily change the input value without modifying the entire prompt structure.
    3. **Readability**: The prompt template provides a clear and well-defined structure for the prompt, which is maintained separately from the business logic. This separation makes it easier for other developers to understand the purpose of the prompt and how it interacts with the model. Decoupling the prompt creation logic from the core functionality enhances the overall clarity and maintainability of the code.
    4. **Flexibility**: You can customize the template to suit your specific use case or domain requirements. This flexibility enables you to adapt the prompt to different scenarios without rewriting the entire prompt logic.
    5. **Scalability**: Langchain’s templating system allows for easy management of multiple prompts, which can be reused and adapted as needed without duplicating code.
    6. **Maintainability**: The prompt template can be updated in a single place, reducing the risk of errors and making it easier to maintain.

### 2.1 Zero-shot Prompt
- Zero-shot prompt is the simplest type of prompt. It provides no examples to the model, just the instruction. 
- You can phrase the instruction as a question. i.e: *"Explain the concept of Generative AI."*
- You can also give the model a 'role'. i.e: *"You are a Data Scientist. Explain the concept of Generative AI."*

In [ ]:
# Define the prompt template
prompt = PromptTemplate(
  input_variables = ["country"],
  template = "What is the capital of {country}?",
)

try:
  # In order to use Langchain, we need to instantiate Langchain extension
  lc_llm_model = WatsonxLLM(model=llm_model)
  
  # Define a chain based on model and prompt
  chain = LLMChain(llm=lc_llm_model, prompt=prompt)

  # Getting predictions
  countries = ["France", "Japan", "Australia"]
  for country in countries:
    response = chain.run(country)
    print(prompt.format(country=country) + " = " + response.capitalize())
    sleep(0.5)
except Exception as e:
  print(e)

### 2.2 Few-shot Prompt
- Few-shot prompt is giving the model a few examples to figure out how to handle similar task in the future.
- It helps the model understand the task better.

In [ ]:
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

# Few -shot examples
examples = [
    {"input": "What is the capital of Sweden?", "output": "Stockholm"},
    {"input": "What is the capital of Malaysia?", "output": "Kuala Lumpur"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [('human', '{input}'), ('ai', '{output}')]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        #('system', 'You are a helpful AI Assistant'),
        few_shot_prompt,
        ('human', '{input}'),
    ]
)

In [ ]:
try:
  # In order to use Langchain, we need to instantiate Langchain extension
  lc_llm_model = WatsonxLLM(model=llm_model)
  
  # Define a chain based on model and prompt
  chain = LLMChain(llm=lc_llm_model, prompt=final_prompt)

  # Getting predictions
  countries = ["France", "Japan", "Australia"]
  for country in countries:
    prompt = f"What is the capital of {country}?"
    print(prompt)
    response = chain.run(prompt)
    print(response)
    #print(prompt.format(country=country) + " = " + response.capitalize())
    sleep(0.5)
except Exception as e:
  print(e)

In [ ]:
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

# Few -shot examples
examples = [
    {"input": "What is the capital of Sweden?", "output": "The capital of Sweden is Stockholm"},
    {"input": "What is the capital of Malaysia?", "output": "The capital of Malaysia is Kuala Lumpur"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [('human', '{input}'), ('ai', '{output}')]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        #('system', 'You are a helpful AI Assistant'),
        few_shot_prompt,
        ('human', '{input}'),
    ]
)

In [ ]:
try:
  # In order to use Langchain, we need to instantiate Langchain extension
  lc_llm_model = WatsonxLLM(model=llm_model)
  
  # Define a chain based on model and prompt
  chain = LLMChain(llm=lc_llm_model, prompt=final_prompt)

  # Getting predictions
  countries = ["France", "Japan", "Australia"]
  for country in countries:
    prompt = f"What is the capital of {country}?"
    print(prompt)
    response = chain.run(prompt)
    print(response)
    #print(prompt.format(country=country) + " = " + response.capitalize())
    sleep(0.5)
except Exception as e:
  print(e)

## 3. Sequential Prompts using Simple Sequential Chain
- By using Simple Sequential Chain in LangChain, you can easily chain multiple prompts to create sequential prompts.
- Prompt chaining, also known as Sequential prompts, enables the response to one prompt to become the input for the next prompt in the sequence.
- Each subsequent prompt is informed by the AI's previous response, creating a chain of interactions that progressively refines the model's output.
- Reference: [SimpleSequentialChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SimpleSequentialChain.html)

In [ ]:
# Create two sequential prompts 
pt1 = PromptTemplate(input_variables=["topic"], template="Generate a random question about {topic}: Question: ")
pt2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

In [ ]:
# Instantiate 2 models (Note, these could be different models depending on use case)
# Note the .to_langchain() method which returns a WatsonxLLM wrapper, like above.
model_1 = Model(
    model_id="google/flan-ul2",
    params=params,
    credentials=creds,
    project_id=project_id
).to_langchain()

model_2 = Model(
    model_id="google/flan-ul2",
    credentials=creds,
    project_id=project_id
).to_langchain()

In [ ]:
# Construct the sequential chain
prompt_to_model_1 = LLMChain(llm=model_1, prompt=pt1)
prompt_to_model_2 = LLMChain(llm=model_2, prompt=pt2)
qa = SimpleSequentialChain(chains=[prompt_to_model_1, prompt_to_model_2], verbose=True)

In [ ]:
# Run our chain with the topic: "an animal"
# Play around with providing different topics to see the output. eg. cars, the Roman empire
try:
  qa.run("an animal")
except Exception as e:
  print(e)